In [ ]:
import psycopg2
conn = psycopg2.connect("host= dbname= user= port= password=")
conn.autocommit = True
cur = conn.cursor()

In [ ]:
cur.execute("SELECT * FROM customer_keywords WHERE id>178")
list_from_db = cur.fetchall()
import pandas as pd
df = pd.DataFrame(list_from_db, columns=["id", "customer_id", "name", "forms", "blacklist", "blacklist_forms"])

In [ ]:
ids = df['id'].values.tolist()
names = df['name'].values.tolist()

In [ ]:
import config
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def make_all(x):
    res = []
    wrds = x.split()
    for langtype in ['nomn', 'gent', 'datv', 'accs', 'ablt', 'loct']:
        newforms = []
        try:
            for w in wrds:
                w = w.strip()
                w_morph = morph.parse(w)[0]
                inflected = w_morph.inflect({langtype})
                if inflected:
                    w_newform = inflected.word
                    if len(w_newform) > 0:
                        newforms.append(w_newform)
                elif len(wrds) > 1:
                    newforms.append(w)
            if len(newforms) > 0:
                new_phrase = ' '.join(newforms)
                res.append(new_phrase)
        except Exception as ex:
            print(ex)
    return list(set(res))

In [ ]:
def make_forms(x):
    forms_arr = make_all(x)
    if len(forms_arr) > 0:
        forms = ','.join(forms_arr)
        forms = '{' + forms + '}'
        return forms
    else:
        return '{}'

In [ ]:
forms = []
i = 0
while i<len(ids):
    forms.append(make_forms(names[i]))
    i = i+1

In [ ]:
i = 0
while i<len(ids):
    cur.execute("UPDATE customer_keywords SET forms='{1}' WHERE id={0}".format(ids[i], forms[i]))
    i = i+1

In [ ]:
cur.close()
conn.close()